# Clasificador binario
Vamos a utilizar Spacy y scikit-learn para clasificar con conjunto de tweets en español como positivos/negativos (análisis de sentimientos)

## Carga y preparación de los datos

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

# Leemos los datos
df = pd.read_csv('tweets_all.csv', index_col=None)

df.head()

In [ ]:
df.info()

In [ ]:
df.polarity.value_counts()

Tenemos 1514 tweets, de los cuales hay 637 positivos y 474 negativos. El resto son neutros o no tienen polaridad clasificada.
Vamos a entrenar sólo con los positivos y negativos para utilizar un clasificador binario

In [ ]:
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

In [ ]:
df.polarity.value_counts()

In [ ]:
df.info()

In [ ]:
df.sample(5)

Quitamos las columnas que no usamos.

## Limpieza de texto
Hacemos un pequeño pre-procesado del texto antes de extraer las características:  
- Quitamos las menciones y las URL del texto porque no aportan valor para el análisis de sentimientos.
- Los hashtag sí que pueden aportar valor así que simplemente quitamos el #.
- Quitamos los signos de puntuación y palabras menores de 3 caracteres.
- Por último quitamos todos los símbolos de puntuación del texto (que forman parte de un token).
- Lematizamos el texto y lo guardamos en otra columna para comparar resultados del clasificador. 

In [ ]:
import re, string, spacy
nlp=spacy.load('es_core_news_md')

In [ ]:
#lista de stop-words específicos de nuestro corpus (aproximación)
stop_words = ['unos', 'unas', 'algún', 'alguna', 'algunos', 'algunas', 'ese', 'eso', 'así']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, lemas=False):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Si lemas=True extraemos el lema, si no dejamos en minúsculas solamente.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lemma_.lower() if lemas else tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', tok) for tok in tokens if not (tok in stop_words) and len(tok)>2]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text
    

Probamos el funcionamiento de estas funciones sobre un tweet de ejemplo:

In [ ]:
print('Original:\n',df.content[702])
print('\nLimpiado:\n',clean_text(df.content[702]))
print('\nLematizado:\n',clean_text(df.content[702], lemas=True))

Aplicamos limpieza a todos los tweets del DataFrame y creamos columna nueva con los lemas

In [ ]:
df["limpio"]=df['content'].apply(clean_text)

In [ ]:
#Quitamos tweets vacíos después de la limpieza
df=df[df.limpio!='']

In [ ]:
df.info()

In [ ]:
df["lemas"]=df.content.apply(clean_text, lemas=True)

In [ ]:
df.head()

In [ ]:
#Contamos el nº de palabras por tweet
df['words'] = [len(t.split(' ')) for t in df.limpio]

In [ ]:
df.describe()

### Clasificador
Vamos a usar la librería scikit-learn para aplicar un clasificador binario sobre la polaridad usando una extracción de características Bag-of-Words (BoW)

Primero dividimos en conjunto de entrenamiento y test.

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test = train_test_split(df['limpio'], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
print('Primera entrada de train:\n', X_train.iloc[0])
print('Polaridad:', y_train.iloc[0])
print('\nX_train shape:', X_train.shape)
print('\nX_test shape:', X_test.shape)

### Modelos basados en el topic vector space
Aplicamos un modelo LSA a la salida del TF-IDF y lo usamos como entrada a nuestro clasificador

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline 

In [ ]:
vect = TfidfVectorizer(ngram_range=(1,2))
svd = TruncatedSVD(n_components=750)

red_dim = make_pipeline(vect, svd, Normalizer(copy=False))
#Entrenamos el modelo con el conjunto de train
lsa_train = red_dim.fit_transform(X_train)

In [ ]:
lsa_train.shape

In [ ]:
lsa_test = red_dim.transform(X_test)
lsa_test.shape

### Entrenamiento
Probamos con varios modelos de clasificación sobre el dataset reducido

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

modelos = [('Logistic Regression', LogisticRegression(solver='liblinear')),
           ('Naive Bayes', GaussianNB()),
           ('Linear SVM', SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5)),
           ('RFB SVM', SVC(gamma='scale', C=2))]

for m, clf in modelos:
    print('Modelo {} con características LSA 750 dims'.format(m))
    #entrenamos sobre train
    clf.fit(lsa_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(lsa_test)
    print(classification_report(y_test, prediccion))
